In [1]:
import os
import time

import numpy as np
import xarray as xr

from dask.distributed import Client, LocalCluster
from scipy.interpolate import interp1d

In this notebook, we plot the meridional overturning circulation in isentropic coordinates &mdash; that is, where zonal averages are taken along surfaces of constant potential temperature. We follow the formulation given in Held and Schneider (1999). Recall that the potential temperature is
$$\theta = T \left(\frac{p_{\textrm{surf}}}{p}\right)^{0.286}$$
Then we define the mean meridional mass flux
$$M(\theta) = -g^{-1}\overline{\frac{\partial p}{\partial \theta}v}$$
where the overbar now indicates a zonal average along the corresponding isentropic surface. Taking such zonal averages will be the bulk of the practical challenge in computing the isentropic circulation. With the mass flux in hand, we can compute the isentropic streamfunction according to
$$\frac{\partial \psi}{\partial \theta} = 2\pi a \cos\vartheta M(\theta)$$
where $a$ is the radius of the Earth and $\vartheta$ is the latitude.

We begin by loading the ERA5 data. The only variables we really need are $v$ and $T$. For a more detailed explanation of the code below, see the analogous cells in `process.ipynb`.

In [2]:
scratch_dir = '/glade/scratch/dconnell'
os.system(f'rm -rf {scratch_dir}/dask-worker-space')

client = Client(LocalCluster(n_workers=20, local_directory=scratch_dir))
display(client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dconnell/proxy/42060/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dconnell/proxy/42060/status,Workers: 20
Total threads: 20,Total memory: 300.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40778,Workers: 20
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dconnell/proxy/42060/status,Total threads: 20
Started: Just now,Total memory: 300.00 GiB
Comm: tcp://127.0.0.1:33001,Total threads: 1
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/dconnell/proxy/33327/status,Memory: 15.00 GiB
Nanny: tcp://127.0.0.1:44789,


In [3]:
era_dir = '/gpfs/fs1/collections/rda/data/ds633.1/e5.moda.an.pl'
years = [int(x) for x in sorted(os.listdir(era_dir))]

ds_fnames = []
for year in years:
    year_dir = f'{era_dir}/{year}'
    fnames = [x for x in os.listdir(year_dir) if x.endswith('nc')]
    
    for name in ['v', 'T']:
        fname = [x for x in fnames if f'_{name.lower()}.' in x][0]
        ds_fnames.append(f'{year_dir}/{fname}')
        
with xr.open_mfdataset(ds_fnames, combine='by_coords', chunks={'time' : 1}) as ds:
    ds = ds.rename({
        'level' : 'pressure',
        'V' : 'v',
    }).drop('utc_date')
    display(ds)
    
    p = (100 * ds['pressure']).assign_attrs(units='Pa')
    lat = (np.pi * ds['latitude'] / 180).assign_attrs(units='radians_north')    
    ds = ds.assign_coords(pressure=p, latitude=lat)
    
    p, lat = ds['pressure'], ds['latitude']
    v, T = ds['v'], ds['T']

,Array,Chunk
Bytes,73.84 GiB,146.54 MiB
Shape,"(516, 37, 721, 1440)","(1, 37, 721, 1440)"
Count,1075 Tasks,516 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,73.84 GiB,146.54 MiB
Shape,"(516, 37, 721, 1440)","(1, 37, 721, 1440)"
Count,1075 Tasks,516 Chunks
Type,float32,numpy.ndarray


With the pressure and temperature loaded, we can compute the $\theta$ field.

In [4]:
p_surf = p[-1].item()
theta = T * ((p_surf / p) ** (0.286))

Now, we need to be able to interpolate multidimensional fields in pressure coordinates to theta coordinates. We need to iterate through each `(time, lat, lon)` point and interpolate the profile of a given variable against the $\theta$ profile at that point. Doing so with an explicit loop would be slow, but `xr.apply_ufunc` provides a means to do this interpolation more efficiently. Below we provide such functionality.

In [5]:
def to_theta_coordinates(theta, data, theta_grid):
    def ufunc(x, y):
        return interp1d(x, y, fill_value='extrapolate')(theta_grid)
    
    return xr.apply_ufunc(
        ufunc, theta, data,
        input_core_dims=[['pressure'], ['pressure']],
        output_core_dims=[['theta']],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[np.float64],
        dask_gufunc_kwargs={'output_sizes' : {'theta' : len(theta_grid)}}
    ).assign_coords(theta=theta_grid)

Now, we'll interpolate the $v$ and $p$ fields to isentropic coordinates. Following Figure 1 of Held and Schneider (1999), we will consider the potential temperature range $240\mathrm{K}$ &mdash; $360\mathrm{K}$.

In [6]:
theta_grid = np.linspace(240, 360, 50)
v_iso = to_theta_coordinates(theta, v, theta_grid)
p_iso = to_theta_coordinates(theta, p, theta_grid)

Next, we can compute the mass flux $M$ as a function of potential temperature.

In [7]:
M = -(p_iso.differentiate('theta') * v_iso).mean('longitude') / 9.8

Finally, we'll assemble a dataset with the $v$, $p$, and $M$ data and save it to disk. We take $v$ and $p$ only in the zonal mean, to make the size manageable. As in `process.ipynb`, it is in this cell that all the deferred computations are carried out, so some waiting is required.

In [8]:
ds = xr.Dataset({
    'v' : v_iso,
    'p' : p_iso,
    'M' : M
}).mean('longitude')

lat = (180 * ds['latitude'] / np.pi).assign_attrs(units='degrees_north')
ds = ds.assign_coords(latitude=lat)

display(ds)
ds.to_netcdf('/glade/work/dconnell/brewson/iso.nc')

<xarray.Dataset>
Dimensions:   (latitude: 721, time: 516, theta: 50)
Coordinates:
  * latitude  (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time      (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
  * theta     (theta) float64 240.0 242.4 244.9 247.3 ... 355.1 357.6 360.0
Data variables:
    v         (time, latitude, theta) float64 dask.array<chunksize=(1, 721, 50), meta=np.ndarray>
    p         (time, latitude, theta) float64 dask.array<chunksize=(1, 721, 50), meta=np.ndarray>
    M         (time, latitude, theta) float64 dask.array<chunksize=(1, 721, 50), meta=np.ndarray>